In [1]:
import pandas as pd
import numpy as np
from numpy import random

# **Chuẩn bị dữ liệu ranking**

In [2]:
dir = '/content/drive/MyDrive/20212/THDL/data/after_matching/data_final.csv'

src_df = pd.read_csv(dir)

In [3]:
src_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7060 entries, 0 to 7059
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Image              7056 non-null   object 
 1   new_name           7054 non-null   object 
 2   new_add            7057 non-null   object 
 3   ID                 7060 non-null   int64  
 4   Cluster            572 non-null    float64
 5   Website            7060 non-null   object 
 6   City               7060 non-null   object 
 7   Hotel name         7060 non-null   object 
 8   Url                7060 non-null   object 
 9   Address            7060 non-null   object 
 10  Stars              7060 non-null   float64
 11  Price              7060 non-null   int64  
 12  Rating             7060 non-null   float64
 13  Number of reviews  7060 non-null   int64  
 14  Reviews            4735 non-null   object 
 15  Facilities         6997 non-null   object 
 16  Description        6291 

In [4]:
#Chỉ lấy những thuộc tính quan trọng
cols = ['Hotel name', 'Stars', 'Price', 'Rating', 'Number of reviews']
hotels_df = src_df[cols]

In [5]:
#Tạo cột hotel_id
hotels_df = hotels_df.reset_index()
hotels_df.rename(columns = {'index':'hotel_id'}, inplace = True)
hotels_df['hotel_id'] = hotels_df['hotel_id'] + 1

In [6]:
hotels_df.describe()

,hotel_id,Stars,Price,Rating,Number of reviews
count,7060.000000,7060.000000,7.060000e+03,7060.000000,7060.000000
mean,3530.500000,2.264943,1.494647e+06,7.084873,176.334419
std,2038.190783,1.578703,5.234637e+06,3.171041,414.244333
min,1.000000,0.000000,2.080400e+04,0.000000,0.000000
25%,1765.750000,1.000000,4.100000e+05,7.300000,4.000000
50%,3530.500000,3.000000,6.500000e+05,8.400000,36.500000
75%,5295.250000,3.000000,1.100000e+06,8.900000,164.000000
max,7060.000000,5.000000,3.391889e+08,10.000000,6441.000000


Do có những bản ghi bị missing value ở trường 'Stars', 'Rating' hay 'Number of reviews' --> Ta sẽ random giá trị trong khoảng

*   Stars: 1-4.5
*   Rating: 6.0-9.0
*   Reviews: 10-100



In [7]:
x = random.choice([1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5])
hotels_df['Stars'] = np.where(hotels_df['Stars'] == 0, x, hotels_df['Stars'])
y = random.randint(60,90)/10
hotels_df['Rating'] = np.where(hotels_df['Rating'] == 0, y, hotels_df['Rating'])
z = random.randint(10,100)
hotels_df['Number of reviews'] = np.where(hotels_df['Number of reviews'] == 0, z, hotels_df['Number of reviews'])

In [8]:
hotels_df.describe()

,hotel_id,Stars,Price,Rating,Number of reviews
count,7060.000000,7060.000000,7.060000e+03,7060.000000,7060.000000
mean,3530.500000,2.639731,1.494647e+06,8.347394,178.302833
std,2038.190783,1.102766,5.234637e+06,0.960894,413.431722
min,1.000000,1.000000,2.080400e+04,1.000000,1.000000
25%,1765.750000,1.500000,4.100000e+05,8.100000,13.000000
50%,3530.500000,3.000000,6.500000e+05,8.400000,36.500000
75%,5295.250000,3.000000,1.100000e+06,8.900000,164.000000
max,7060.000000,5.000000,3.391889e+08,10.000000,6441.000000


# **Ranking**

Một số tiêu chí ranking:

1.   Stars càng nhiều càng tốt --> Hệ số stars dương
1.   Giá càng cao thì rank càng thấp --> Hệ số giá âm
1.   Rating càng cao và càng nhiều reviews càng tốt --> Hệ số dương




In [41]:
new_al = []
for i in range(len(hotels_df['hotel_id'])):
  alpha = 0
  if hotels_df['Stars'][i] <= 2.0:
    alpha += 0.25
  if hotels_df['Rating'][i] <= 6:
    alpha += 0.75
  if hotels_df['Number of reviews'][i] <= 10:
    alpha += 0.5
  if hotels_df['Price'][i]>3000000:
    alpha += 1
  if hotels_df['Stars'][i] >= 4.5 and hotels_df['Rating'][i] >= 9.5 and hotels_df['Price'][i] <= 500000:
    alpha -= 1
  
  new_al.append(alpha)

hotels_df['alpha'] = new_al

In [42]:
m, n, p, q = 4, 0.25, 40, 2 
a, b, c, d = 1/(m*max(hotels_df['Rating'])), -1/(n*max(hotels_df['Price'])), 1/(p*max(hotels_df['Number of reviews'])), 1/(q*max(hotels_df['Stars'])),
hotels_df['overall_score'] = a*hotels_df['Rating'] + b*hotels_df['Price'] + c*hotels_df['Number of reviews'] + d*hotels_df['Stars'] - alpha

ranking_df = hotels_df.sort_values('overall_score', ascending=False)

In [43]:
ranking_df

,hotel_id,Hotel name,Stars,Price,Rating,Number of reviews,alpha,overall_score
4541,4542,Căn Hộ Star Beach (Star Beach Apartment),5.0,445886,9.6,127,-1.00,0.235235
6816,6817,Khách sạn Mường Thanh Luxury Viễn Triều Nha Trang,5.0,601093,9.6,77,0.00,0.233210
4960,4961,Khách Sạn The Oriental Jade (The Oriental Jade...,5.0,982461,9.5,1856,0.00,0.233118
118,119,Khách sạn Five Star Westlake (Hotel & Serviced...,5.0,1476000,10.0,1,0.50,0.232598
6801,6802,Khách sạn Vesna Nha Trang,5.0,667883,9.6,21,0.00,0.232205
...,...,...,...,...,...,...,...,...
5805,5806,Khách sạn Villathol Đà Lạt,1.0,50000000,8.7,53,1.25,-0.771936
5722,5723,Chou Hostel,1.0,50000000,8.2,13,1.25,-0.784591
5819,5820,Khách sạn Len's Mini,1.0,50000000,7.7,96,1.25,-0.796769
6154,6155,Signature Royal Cruise,4.0,100000000,8.2,3,1.50,-1.074272


In [44]:
ranking_df = ranking_df.reset_index(drop=True).reset_index()
ranking_df.drop({'overall_score', 'alpha'}, axis=1, inplace=True)
ranking_df.rename({'index':'Rank'}, axis=1, inplace=True)
ranking_df['Rank'] = ranking_df['Rank'] + 1

,Rank,hotel_id,Hotel name,Stars,Price,Rating,Number of reviews
0,1,4542,Căn Hộ Star Beach (Star Beach Apartment),5.0,445886,9.6,127
1,2,6817,Khách sạn Mường Thanh Luxury Viễn Triều Nha Trang,5.0,601093,9.6,77
2,3,4961,Khách Sạn The Oriental Jade (The Oriental Jade...,5.0,982461,9.5,1856
3,4,119,Khách sạn Five Star Westlake (Hotel & Serviced...,5.0,1476000,10.0,1
4,5,6802,Khách sạn Vesna Nha Trang,5.0,667883,9.6,21
...,...,...,...,...,...,...,...
7055,7056,5806,Khách sạn Villathol Đà Lạt,1.0,50000000,8.7,53
7056,7057,5723,Chou Hostel,1.0,50000000,8.2,13
7057,7058,5820,Khách sạn Len's Mini,1.0,50000000,7.7,96
7058,7059,6155,Signature Royal Cruise,4.0,100000000,8.2,3


# **Lưu file ranking đầy đủ**

In [45]:
demo = src_df.copy(deep=True)

In [46]:
#Tạo cột hotel_id
demo = demo.reset_index()
demo.rename(columns = {'index':'hotel_id'}, inplace = True)
demo['hotel_id'] = demo['hotel_id'] + 1

In [47]:
x = random.choice([1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5])
demo['Stars'] = np.where(demo['Stars'] == 0, x, demo['Stars'])
y = random.randint(60,90)/10
demo['Rating'] = np.where(demo['Rating'] == 0, y, demo['Rating'])
z = random.randint(10,100)
demo['Number of reviews'] = np.where(demo['Number of reviews'] == 0, z, demo['Number of reviews'])

In [48]:
new_al = []
for i in range(len(demo['hotel_id'])):
  alpha = 0
  if demo['Stars'][i] <= 2.0:
    alpha += 0.25
  if demo['Rating'][i] <= 6:
    alpha += 0.75
  if demo['Number of reviews'][i] <= 10:
    alpha += 0.5
  if demo['Price'][i]>3000000:
    alpha += 1
  if demo['Stars'][i] >= 4.5 and demo['Rating'][i] >= 9.5 and demo['Price'][i] <= 500000:
    alpha -= 1
  
  new_al.append(alpha)

demo['alpha'] = new_al

In [49]:
m, n, p, q = 4, 0.25, 40, 2 
a, b, c, d = 1/(m*max(demo['Rating'])), -1/(n*max(demo['Price'])), 1/(p*max(demo['Number of reviews'])), 1/(q*max(demo['Stars'])),
demo['overall_score'] = a*demo['Rating'] + b*demo['Price'] + c*demo['Number of reviews'] + d*demo['Stars'] - alpha

ranking = demo.sort_values('overall_score', ascending=False)

In [50]:
ranking = ranking.reset_index(drop=True).reset_index()
ranking.drop({'overall_score', 'alpha'}, axis=1, inplace=True)
ranking.rename({'index':'Rank'}, axis=1, inplace=True)
ranking['Rank'] = ranking['Rank'] + 1

In [53]:
ranking.to_csv('/content/drive/MyDrive/20212/THDL/data/after_matching/ranking_hotels.csv', index=False, encoding='utf8')